# Battle of the Neighborhoods - Coursera Capstone Project 


##### Author: Aaron E

## Introduction 

#### A local BBQ chain would like to expand to a new city. The prospect city is Toronto Canada. Toronto Canada is a rapidly developing city and provides various business oppourtunities. The city has become a major hub for tourist and businesses alike. Becuase of the growing compitetion in the area the stakeholders would like a detailed review of all the compitetion in the Toronto area and a detailed analysis of the surrounding areas.

#### We will be using Foursquare API to explore the restaurants in the Toronto area and to explore the most common venues while also using a list of zip codes from a wikipedia page to explore the neighborhoods.

#### from the data we will be able to assess the compitetion, and weather or not the area already has a sucessful BBQ restaurant. 

#### First lets explorea few choice boroughs in Toronoto, finding the most popular venues 

In [2]:
# Imported needed libraries
# importing old libraries (also used in the two first parts)
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


Read in the html as a Pandas Dataframe 

In [20]:
#get html from wiki page and create soup object
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
canada_df = df 
canada_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Now we must clean up the data

Ignor the cells with bourough not assigned

In [21]:
# Ignore cells with Bourough not assigned
canada_df = canada_df[canada_df["Borough"] != "Not assigned"]

Group the data

In [23]:
df_toronto = canada_df.groupby(["Postal Code", "Borough"], as_index=False).agg(','.join)
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Replace mising values in Neighborhood with Borough if Neighborhood is not assigned

In [24]:
df_toronto['Neighbourhood'] = np.where(df_toronto['Neighbourhood'] == 'Not assigned',
                                              df_toronto['Borough'], df_toronto['Neighbourhood'])

Lets check the data shape

In [25]:
df_toronto.shape

(103, 3)

Lets get the geosatial data and add it to our dataframe

In [26]:
geospatial_url = "https://cocl.us/Geospatial_data"
geospatial_data = pd.read_csv(geospatial_url)

In [27]:
merged_data = pd.merge(df_toronto, geospatial_data, on='Postal Code')

In [31]:
df_toronto = merged_data

In [37]:
df_toronto.head(100)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
95,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
96,M9L,North York,Humber Summit,43.756303,-79.565963
97,M9M,North York,"Humberlea, Emery",43.724766,-79.532242
98,M9N,York,Weston,43.706876,-79.518188


## Lets see a map of Toronto with all the neighborhood 

In [33]:
address = "Toronto, Ontario"
geolocator = Nominatim(user_agent="ny_explorer", timeout=30)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.6534817, -79.3839347.


In [43]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## This client is interested in a more populated part of the city, below we will anaylize Downtown Toronto.

In [36]:
# Credentials for Foursquare API
# @hidden_cell
CLIENT_ID = 'S0HWGHYZKXRMZUK0JN5FM1R2RDBN5XUYURRAEL01MIJCJYP5' # your Foursquare ID
CLIENT_SECRET = '4ILG5R1AX5H3JRJ3I5XGIOGB4EOZJBDRRSHQQKMH4NP344DE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S0HWGHYZKXRMZUK0JN5FM1R2RDBN5XUYURRAEL01MIJCJYP5
CLIENT_SECRET:4ILG5R1AX5H3JRJ3I5XGIOGB4EOZJBDRRSHQQKMH4NP344DE


In [58]:
down_town_data = df_toronto[df_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
james_town_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


In [59]:
james_town_data["Neighbourhood"].count()

19

# 19 neighborhoods in Downtown Toronoto

### Map of Downtown Toronto

In [44]:
address_scar = 'Downtown Toronto'
latitude_scar = 43.667967
longitude_scar = -79.3676675
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.667967, -79.3676675.


In [46]:
map_downtown = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(down_town_data['Latitude'], down_town_data['Longitude'], down_town_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_downtown)  
    
map_downtown

### Top venues in Downtown Toronto

In [49]:
down_town_data.loc[1, "Neighbourhood"]

'St. James Town, Cabbagetown'

In [51]:
neighbourhood_lat = down_town_data.loc[1,"Latitude"] # Neighbourhood lat value
neighbourhood_long = down_town_data.loc[1, "Longitude"] # Neighbourhood long value
neighbourhood_name = down_town_data.loc[1, "Neighbourhood"] # neighbourhoods name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_lat, 
                                                               neighbourhood_long))

Latitude and longitude values of St. James Town, Cabbagetown are 43.667967, -79.3676753.


Setting a radius

In [118]:
LIMIT = 500
radius = 15000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,
                                                                                                                           latitude_scar, longitude_scar, 
                                                                                                                           VERSION, radius, LIMIT)


In [119]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f21e8ffd46d721c653d9ccb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 235,
  'suggestedBounds': {'ne': {'lat': 43.80296713500013,
    'lng': -79.18138497979078},
   'sw': {'lat': 43.532966864999864, 'lng': -79.55395002020921}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb63b82941ad13a5e181fe3',
       'name': 'Riverdale Park West',
       'location': {'address': '500 Gerrard St.',
        'crossStreet': 'at River St.',
        'lat': 43.66604827676354,
        'lng': -79.36094117493359,
        'labeledLatLngs': [{'label': 'display'

In [120]:
# Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [121]:
# structure and clean data into pandas dataframe 
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Riverdale Park West,Park,43.666048,-79.360941
1,Riverdale Park East,Park,43.669951,-79.355493
2,House on Parliament,Gastropub,43.663646,-79.367854
3,Zakkushi ざっ串,Japanese Restaurant,43.663618,-79.370670
4,Riverdale Farm,Farm,43.666850,-79.361376
...,...,...,...,...
95,Lake Ontario,Lake,43.638945,-79.379665
96,Harbourfront Centre,Performing Arts Venue,43.638556,-79.383190
97,The Moonbean Cafe,Café,43.654147,-79.400182
98,Rodney's Oyster House,Seafood Restaurant,43.644975,-79.396587


In [122]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


# 100 venues have been found in the surrounding areas of St.JamesTown in Downtown Toronto, we will analyize each one. 

In [123]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [124]:
downtown_neighborhood_venues = getNearbyVenues(names=down_town_data['Neighbourhood'],
                                   latitudes=down_town_data['Latitude'],
                                   longitudes=down_town_data['Longitude']
                                  )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [125]:
downtown_neighborhood_venues.head(15)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
5,"St. James Town, Cabbagetown",43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
6,"St. James Town, Cabbagetown",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
7,"St. James Town, Cabbagetown",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
8,"St. James Town, Cabbagetown",43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant
9,"St. James Town, Cabbagetown",43.667967,-79.367675,Merryberry Cafe + Bistro,43.666630,-79.368792,Café


In [85]:
downtown_neighborhood_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,62,62,62,62,62,62
Christie,17,17,17,17,17,17
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [126]:
print('There are {} uniques categories.'.format(len(downtown_neighborhood_venues['Venue Category'].unique())))

There are 209 uniques categories.


# Count of how many venues are in each neighborhood; there are 209 unique categories in Downtown Toronto

# Next we will see the type of venues in each neighborhood

In [127]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouped neighborhoods by mean of frewuency of ocurrence of each category 

In [128]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0.000811,0.000811,0.000811,0.001622,0.001622,0.001622,0.01541,0.001622,0.004055,...,0.000811,0.000811,0.002433,0.011354,0.000811,0.003244,0.006488,0.000811,0.000811,0.005677
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Lets simplify and find the top 10 venues 

In [129]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [130]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Seafood Restaurant,Gym,Bakery
1,1,Yoga Studio,Women's Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


# The most popular venues in the neighborhood of JamesTown, Rosedale, and Cabbage Town in the borough of Downtown Toronto is showed above.

# The most popular venue is a coffee shop in Rosedale, and a Yoga Studio in Jamestown/Cabbage Town.

# Lets now explore another area

In [137]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Map of Scarborough

In [138]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [139]:
map_scarborough = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarborough)  
    
map_scarborough

### Top venues in the neighborhood "Scarborough Village"

In [140]:
scarborough_data.loc[5, "Neighbourhood"]

'Scarborough Village'

In [141]:
neighbourhood_lat = scarborough_data.loc[5,"Latitude"] # Neighbourhood lat value
neighbourhood_long = scarborough_data.loc[5, "Longitude"] # Neighbourhood long value
neighbourhood_name = scarborough_data.loc[5, "Neighbourhood"] # neighbourhoods name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_lat, 
                                                               neighbourhood_long))

Latitude and longitude values of Scarborough Village are 43.7447342, -79.23947609999999.


In [153]:
LIMIT = 500
radius = 15000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,
                                                                                                                           latitude_scar, longitude_scar, 
                                                                                                                           VERSION, radius, LIMIT)


In [154]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f21eb5272cee757fc3ef7de'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 233,
  'suggestedBounds': {'ne': {'lat': 43.908077135000134,
    'lng': -79.07116438641437},
   'sw': {'lat': 43.63807686499987, 'lng': -79.44438361358563}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c5c9bee94fd0f47cb30c945',
       'name': 'The Roti Hut',
       'location': {'address': '351 Pitfield Rd',
        'crossStreet': 'McCowan Rd',
        'lat': 43.78727725968595,
        'lng': -79.25872401779647,
        'labeledLatLngs': [{'label': 'display',
       

In [148]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [157]:
# structure and clean data into pandas dataframe 
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Roti Hut,Caribbean Restaurant,43.787277,-79.258724
1,Mona's Roti,Caribbean Restaurant,43.791613,-79.251015
2,CANBE Foods Inc,Indian Restaurant,43.773546,-79.246082
3,The Real McCoy Burgers And Pizza,Burger Joint,43.774081,-79.230496
4,Thomson Memorial Park,Park,43.758891,-79.254500


In [158]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [161]:
scarborough_nieghnorhood_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [162]:
scarborough_nieghnorhood_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [163]:
scarborough_nieghnorhood_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",8,8,8,8,8,8
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5


In [164]:
print('There are {} uniques categories.'.format(len(scarborough_nieghnorhood_venues['Venue Category'].unique())))

There are 56 uniques categories.


# There are 56 unique categories in the borough of Scarborough

In [165]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_nieghnorhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_nieghnorhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Café,...,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant,Women's Store
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


grouped neighborhoods by mean of frequency of occurrence of each category

In [166]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped.head(7)

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Café,...,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant,Women's Store
0,Agincourt,0.0,0.000,0.0,0.000,0.000000,0.0,0.25,0.000,0.00,...,0.0,0.0,0.0,0.0,0.00,0.000,0.000000,0.0,0.0,0.0
1,"Birch Cliff, Cliffside West",0.0,0.000,0.0,0.000,0.000000,0.0,0.00,0.000,0.25,...,0.0,0.0,0.0,0.0,0.25,0.000,0.000000,0.0,0.0,0.0
2,Cedarbrae,0.0,0.125,0.0,0.125,0.125000,0.0,0.00,0.000,0.00,...,0.0,0.0,0.0,0.0,0.00,0.000,0.125000,0.0,0.0,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000,0.0,0.000,0.083333,0.0,0.00,0.000,0.00,...,0.0,0.0,0.0,0.0,0.00,0.000,0.083333,0.0,0.0,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000,0.0,0.000,0.000000,0.0,0.00,0.000,0.00,...,0.0,0.0,0.0,0.0,0.00,0.000,0.000000,0.0,0.0,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000,0.0,0.000,0.000000,0.0,0.00,0.000,0.00,...,0.0,0.0,0.0,0.0,0.00,0.000,0.000000,0.0,0.2,0.0
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000,0.0,0.250,0.000000,0.0,0.00,0.125,0.00,...,0.0,0.0,0.0,0.0,0.00,0.125,0.000000,0.0,0.0,0.0


### top 10 venues per neighboring in Scarborough

In [167]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Lounge,Clothing Store,Women's Store,Department Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment
1,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Women's Store,Hakka Restaurant,Gym,Grocery Store,Gas Station,Fried Chicken Joint
2,Cedarbrae,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Gas Station,Caribbean Restaurant,Fried Chicken Joint,Women's Store,Department Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Pharmacy,Fast Food Restaurant,Noodle House,Thai Restaurant,Gas Station,Bank,Italian Restaurant,Fried Chicken Joint,Chinese Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Home Service,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Pet Store,Chinese Restaurant,Vietnamese Restaurant,Bakery,Department Store,Athletics & Sports,Hakka Restaurant,Gym,Grocery Store
6,"Golden Mile, Clairlea, Oakridge",Bakery,Ice Cream Shop,Intersection,Metro Station,Bus Line,Park,Soccer Field,Bank,Discount Store,Gym
7,"Guildwood, Morningside, West Hill",Mexican Restaurant,Intersection,Bank,Medical Center,Breakfast Spot,Electronics Store,Restaurant,Rental Car Location,Women's Store,Fast Food Restaurant
8,"Kennedy Park, Ionview, East Birchmount Park",Convenience Store,Hobby Shop,Discount Store,Department Store,Coffee Shop,Women's Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment
9,"Malvern, Rouge",Fast Food Restaurant,Women's Store,Vietnamese Restaurant,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint


# Cluster Neighborhoods in Scarborough using k-means

In [170]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scar_data = scarborough_data.drop(14)
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 1, 1, 3, 2])

In [186]:
scarborough_merged = scar_data

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarborough_merged.head(15)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Women's Store,Vietnamese Restaurant,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1,Home Service,Bar,Convenience Store,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Mexican Restaurant,Intersection,Bank,Medical Center,Breakfast Spot,Electronics Store,Restaurant,Rental Car Location,Women's Store,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Convenience Store,Korean Restaurant,Women's Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Gas Station,Caribbean Restaurant,Fried Chicken Joint,Women's Store,Department Store
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Women's Store,Playground,College Stadium,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1,Convenience Store,Hobby Shop,Discount Store,Department Store,Coffee Shop,Women's Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1,Bakery,Ice Cream Shop,Intersection,Metro Station,Bus Line,Park,Soccer Field,Bank,Discount Store,Gym
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,3,American Restaurant,Motel,Home Service,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,2,College Stadium,Skating Rink,General Entertainment,Café,Women's Store,Hakka Restaurant,Gym,Grocery Store,Gas Station,Fried Chicken Joint


In [175]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examing the clusters 

Cluster 1

In [178]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,0,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Gas Station,Caribbean Restaurant,Fried Chicken Joint,Women's Store,Department Store


Cluster 2

In [179]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Fast Food Restaurant,Women's Store,Vietnamese Restaurant,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
1,Scarborough,1,Home Service,Bar,Convenience Store,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
2,Scarborough,1,Mexican Restaurant,Intersection,Bank,Medical Center,Breakfast Spot,Electronics Store,Restaurant,Rental Car Location,Women's Store,Fast Food Restaurant
3,Scarborough,1,Coffee Shop,Convenience Store,Korean Restaurant,Women's Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
5,Scarborough,1,Women's Store,Playground,College Stadium,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
6,Scarborough,1,Convenience Store,Hobby Shop,Discount Store,Department Store,Coffee Shop,Women's Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment
7,Scarborough,1,Bakery,Ice Cream Shop,Intersection,Metro Station,Bus Line,Park,Soccer Field,Bank,Discount Store,Gym
10,Scarborough,1,Indian Restaurant,Pet Store,Chinese Restaurant,Vietnamese Restaurant,Bakery,Department Store,Athletics & Sports,Hakka Restaurant,Gym,Grocery Store
11,Scarborough,1,Sandwich Place,Shopping Mall,Fried Chicken Joint,Breakfast Spot,Middle Eastern Restaurant,Bakery,Auto Garage,Grocery Store,General Entertainment,Convenience Store
13,Scarborough,1,Pizza Place,Pharmacy,Fast Food Restaurant,Noodle House,Thai Restaurant,Gas Station,Bank,Italian Restaurant,Fried Chicken Joint,Chinese Restaurant


Cluster 3 

In [180]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,2,College Stadium,Skating Rink,General Entertainment,Café,Women's Store,Hakka Restaurant,Gym,Grocery Store,Gas Station,Fried Chicken Joint


Cluster 4

In [181]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,3,American Restaurant,Motel,Home Service,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
16,Scarborough,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cluster 5

In [183]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,4,Latin American Restaurant,Breakfast Spot,Lounge,Clothing Store,Women's Store,Department Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment
